<h2>Table of Contents</h2>

In this lab, we train a Convolutional Neural Network with Regular data and Augmented data. The purpose of this lab is to show that the Augmented data improves generalization performance.

Think of a scenario where a drone has to take a picture of an object. The drone is moving and the object can also possibly be moving. When an image is taken we arent always going to get perfect images. The subject may not be perfectly centered in the image or the subject may be rotated in the image. In this case, a model trained on perfectly centered or rotated images won't perform well. This is why we train a model on rotated data so it can perform well on imperfect images.

In this assignment, we will use a dataset of digit images. We will have two models one trained on non rotated digits and one trained on rotated images and then we will test the models on a rotated testing dataset which will be more realistic and robust in terms of our scenario above.

<ul>
<li><a href="#Makeup_Data">Get Some Data</a></li>
<li><a href="#CNN">Convolutional Neural Network</a></li>
<li><a href="#R_training_data">Rotated Training Data</a></li>

In [ ]:
!pip3 install torch torchvision torchaudio

In [ ]:
# Download a Pretrained Model because training takes a long time
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/meet_up/12.02.2020/normal.pt

In [ ]:
# Download a Pretrained Model Trained on Augmented Data because training takes a long time
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/meet_up/12.02.2020/rotated_data.pt

In [ ]:
# Library to Show Images
!pip install Pillow

***Restart the kernel***


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import matplotlib.pylab as plt
import numpy as np
import os

Plot Cost and Accuracy vs Epoch Graph


In [ ]:
def plot_cost_accuracy(checkpoint):
    fig, ax1 = plt.subplots()
    color='tab:red'
    ax1.plot(checkpoint['cost'],color=color)
    ax1.set_xlabel('epoch',color=color)
    ax1.set_ylabel('cost',color=color)
    ax1.tick_params(axis='y',color=color)
    
    ax2=ax1.twinx()
    color='tab:blue'
    ax2.plot(checkpoint['accuracy'],color=color)
    ax2.set_xlabel('epoch',color=color)
    ax1.set_ylabel('accuracy',color=color)
    ax1.tick_params(axis='y',color=color)
    fig.tight_layout()

Define the function <code>show_data</code> to plot out data samples as images.


In [ ]:
def show_data(data_sample):
    plt.imshow(data_sample[0].numpy().reshape(IMAGE_SIZE,IMAGE_SIZE),cmap='gray')
    plt.title('y='+str(data_sample[1]))
    

Plot first 5 misclassified samples 


In [ ]:
def plot_mis_classified(model,dataset):
    count=0
    for x,y in torch.utils.data.DataLoader(dataset=dataset,batch_size=1):
        z = model(x)
        _,yhat = torch.max(z,1)
        if yhat != y:
            show_data((x,y))
            plt.show()
            count +=1
        if count >= 5:
            break

<h2 id="Makeup_Data">Load Data</h2> 


We create a transform object  <code>compose</code> one will resize the image and convert it to a tensor, the second will also rotate the image Randomly rotate the image.


In [ ]:
IMAGE_SIZE=16

compose_rotate = transforms.Compose([transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),transforms.RandomAffine(45),transforms.ToTensor()])

compose = transforms.Compose([transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),transforms.ToTensor()])

In [ ]:
train_dataset_rotate = dsets.MNIST(root='/kaggle/working/',train=True,download=True,transform=compose_rotate)
train_dataset=dsets.MNIST(root='/kaggle/working/',train=True,download=True,transform=compose)

In [ ]:
validation_dataset = dsets.MNIST(root='/kaggle/working/',train=False,download=True,transform=compose_rotate)

In [ ]:
show_data(train_dataset[0])

In [ ]:
train_dataset[0][1]

In [ ]:
show_data(train_dataset_rotate[0])

<h2 id="CNN">Build a Convolutional Neural Network Class</h2>
Build a Convolutional Network class with two Convolutional layers and one fully connected layer. Pre-determine the size of the final output matrix. The parameters in the constructor are the number of output channels for the first and second layers.


In [ ]:
class CNN(nn.Module):
    #constructor
    def __init__(self,out_1=16,out_2=32):
        super(CNN,self).__init__()
        # The reason we start with 1 channel is because we have a single black and white image
        # Channel Width after this layer is 16
        self.cnn1 = nn.Conv2d(in_channels=1,out_channels=out_1,kernel_size=5,padding=2)
        # Channel Width after this layer is 8
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        self.cnn2 = nn.Conv2d(in_channels=out_1,out_channels=out_2,kernel_size=5,stride=1,padding=2)
        # Channel Width after this layer is 4
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # In total we have out_2 (32) channels which are each 4 * 4 in size based on the width calculation above. Channels are squares.
        # The output is a value for each class
        self.fc1 = nn.Linear(out_2*4*4,10)
    #prediction
    def forward(self,x):
        x = self.cnn1(x)
        x = torch.relu(x)
        x=self.maxpool1(x)
        x=self.cnn2(x)
        x=torch.relu(x)
        x=self.maxpool2(x)
        x=x.view(x.size(0),-1)
        x=self.fc1(x)
        return x
    def activations(self,x):
        z1 = self.cnn1(x)
        a1 = torch.relu(z1)
        out=self.maxpool1(a2)
        z2=self.cnn2(out)
        a2=torch.relu(z2)
        out1=self.maxpool2(z2)
        return z1,a1,z2,a2,out1,out

<h2 id="regular_data">Regular Data</h2> 
Define the Convolutional Neural Network Classifier, Criterion function, Optimizer, and Train the Model


In [ ]:
# Create the model object to be trained on regular data using CNN class
model = CNN(out_1=16,out_2=32)

In [ ]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=100)
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset,batch_size=5000)

In [ ]:
from tqdm import tqdm
#location to save data
file_normal = os.path.join(os.getcwd(),'normal.pt')
# All the data we are saving
checkpoint = {
    'epoch':None,
    'model_state_dict':None,
    'optimizer_state_dict':None,
    'loss':None,
    'cost':[],
    'accuracy':[]
}

n_epochs=5
# Size of the testing dataset
N_test = len(validation_dataset)
# Training for the number of epochs we want
for epoch in tqdm(range(n_epochs)):
    cost=0
    for x,y in train_loader:
        # Resets the calculated gradient value, this must be done each time as it accumulates if we do not reset
        optimizer.zero_grad()
        # Makes a prediction on the image
        z=model(x)
        # Calculate the loss between the prediction and actual class
        loss=criterion(z,y)
        # Calculates the gradient value with respect to each weight and bias
        loss.backward()
        # Updates the weight and bias according to calculated gradient value
        optimizer.step()
        
        # Saves the number of epochs we trained for  
        checkpoint['epochs'] = n_epochs
        # Saves the models parameters
        checkpoint['model_state_dict'] = model.state_dict()
        # Saves the optimizers paramters
        checkpoint['optimizer_state_dict'] = optimizer.state_dict()
        # Saves the loss for the last batch so ultimately this will be the loss for the last batch of the last epoch
        checkpoint['loss'] = loss
        # Accumulates the loss
        cost += loss.item()
    # Counter for the correct number of predictions        
    correct = 0
    # For each batch in the validation dataset
    for x_test, y_test in validation_loader:
        # Make a prediction
        z = model(x_test)
        # Get the class that has the maximum value
        _, yhat = torch.max(z.data, 1)
        # Counts the number of correct predictions made
        correct += (yhat == y_test).sum().item()
 
    accuracy = correct / N_test
    print(accuracy)
    # Appends the cost of the epoch to a list
    checkpoint['cost'].append(cost) 
    # Appends the accuracy of the epoch to a list
    checkpoint['accuracy'].append(accuracy)
    # Saves the data in checkpoint to the file location
    torch.save(checkpoint, file_normal) 

<h3 id="Result">Analyze Results</h3> 


In [ ]:
checkpoint_normal = torch.load(os.path.join(os.getcwd(),'normal.pt'))

In [ ]:
plot_cost_accuracy(checkpoint_normal)

In [ ]:
# Using the model parameters we saved we load them into a model to recreate the trained model
model.load_state_dict(checkpoint_normal['model_state_dict'])
# Setting the model to evaluation mode
model.eval()
# Using the helper function plot the first five misclassified samples
plot_mis_classified(model,validation_dataset)

<h2 id="R_training_data">Rotated Training Data</h2> 


In [ ]:
# Create the model object using CNN class
model_r = CNN(out_1=16, out_2=32)
# We create a criterion which will measure loss
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
# Create an optimizer that updates model parameters using the learning rate and gradient
optimizer = torch.optim.SGD(model_r.parameters(), lr = learning_rate)
# Create a Data Loader for the rotated training data with a batch size of 100 
train_loader = torch.utils.data.DataLoader(dataset=train_dataset_rotate, batch_size=100)
# Create a Data Loader for the rotated validation data with a batch size of 5000 
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, batch_size=5000)

In [ ]:
# Location to save data
file_rotated = os.path.join(os.getcwd(), 'rotated_data.pt')

# All the data we are saving
checkpoint={
            # Saving the number of epochs the models was trained for
            'epoch': None,
            # Saving the models parameters which will allow us to recreate the trained model
            'model_state_dict': None,
            # Saving the optimizers parameters
            'optimizer_state_dict': None,
            # Saving the loss on the training dataset for the last batch of the last epoch
            'loss': None,
            # Saving the cost on the training dataset for each epoch
            'cost': [],
            # Saving the accuracy for the testing dataset for each epoch
            'accuracy': []}
            
# Number of epochs to train model            
n_epochs = 10

# Size of the testing dataset
N_test = len(validation_dataset)

# Training for the number of epochs we want
for epoch in tqdm(range(n_epochs)):
    # Variable to keep track of cost for each epoch
    cost = 0
    # For each batch in the training dataset
    for x, y in train_loader:
        # Resets the calculated gradient value, this must be done each time as it accumulates if we do not reset
        optimizer.zero_grad()
        # Makes a prediction on the image
        z = model_r(x)
        # Calculate the loss between the prediction and actual class
        loss = criterion(z, y)
        # Calculates the gradient value with respect to each weight and bias
        loss.backward()
        # Updates the weight and bias according to calculated gradient value
        optimizer.step()
      
        # Saves the number of epochs we trained for  
        checkpoint['epochs'] = n_epochs
        # Saves the models parameters
        checkpoint['model_state_dict'] = model.state_dict()
        # Saves the optimizers paramters
        checkpoint['optimizer_state_dict'] = optimizer.state_dict()
        # Saves the loss for the last batch so ultimately this will be the loss for the last batch of the last epoch
        checkpoint['loss'] = loss
        # Accumulates the loss
        cost+=loss.item()
        
     
    # Counter for the correct number of predictions        
    correct = 0
        
    # For each batch in the validation dataset
    for x_test, y_test in validation_loader:
        # Make a prediction
        z = model_r(x_test)
        # Get the class that has the maximum value
        _, yhat = torch.max(z.data, 1)
        # Counts the number of correct predictions made
        correct += (yhat == y_test).sum().item()
 
    accuracy = correct / N_test
    print(accuracy)
    # Appends the cost of the epoch to a list
    checkpoint['cost'].append(cost) 
    # Appends the accuracy of the epoch to a list
    checkpoint['accuracy'].append(accuracy)
    # Saves the data in checkpoint to the file location
    torch.save(checkpoint, file_rotated) 

In [ ]:
checkpoint_rotated= torch.load(os.path.join(os.getcwd(),'rotated_data.pt'))

In [ ]:
plot_cost_accuracy(checkpoint_rotated)

In [ ]:
# Using the model parameters we saved we load them into a model to recreate the trained model
model_r.load_state_dict(checkpoint_rotated['model_state_dict'])
# Setting the model to evaluation mode
model.eval()
# Using the helper function plot the first five misclassified samples
plot_mis_classified(model_r,validation_dataset)